In [ ]:
# 1. 首先清理和更新系統
!apt-get update
!apt-get install -y wget unzip
!apt-get remove -y chromium-browser chromium-chromedriver
!pip uninstall -y selenium

# 2. 安裝 Chrome
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo "deb http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list
!apt-get update
!apt-get install -y google-chrome-stable

# 3. 安裝必要的 Python 包
!pip install selenium
!pip install webdriver_manager

# 4. 確認 Chrome 版本
!google-chrome --version

# 5. 導入必要的庫
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, urljoin
from webdriver_manager.chrome import ChromeDriverManager
import os
import json

In [ ]:
from google.colab import output
output.clear()  # 清除輸出

# 安裝必要套件
!pip install selenium webdriver_manager
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# 導入所需的庫
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, urljoin
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import json


class FlipclassExamBot:
    def __init__(self, account, password, course_id, answer_time, target_score):
        self.account = account
        self.password = password
        self.course_id = course_id
        self.answer_time = float(answer_time)
        self.target_score = target_score
        self.driver = None
        self.wait = None
        self.answer_page_url = None  # 保存答案頁面的 URL
        self.exam_page_url = None    # 保存考試頁面的 URL

        # JavaScript 代碼作為類的屬性
        self.extract_answers_js = """
        const questions = document.querySelectorAll('.kques-item[data-parentid=""]');
        let output = "";
        questions.forEach((question) => {
            const questionType = question.getAttribute('type');
            const dataId = question.getAttribute('data-id');
            if (questionType === '1') {
                const isMultipleChoice = question.querySelector('input[type="checkbox"]') !== null;
                const selectedOptions = isMultipleChoice
                    ? Array.from(question.querySelectorAll('input[type="checkbox"][checked]'))
                    : Array.from(question.querySelectorAll('input[type="radio"][checked]'));
                const answerTexts = selectedOptions
                    .map(option => {
                        let text = option.closest('label').textContent.trim();
                        return text.replace(/^[A-Z]\.\s*/, '');
                    })
                    .join(', ');
                output += `${dataId} - ${answerTexts || "No correct answer found"}\\n`;
            } else if (questionType === '2') {
                const blankAnswers = Array.from(question.querySelectorAll('span.gap-preview'))
                    .map(blank => blank.textContent.trim())
                    .join(', ');
                output += `${dataId} - ${blankAnswers || "No answers provided"}\\n`;
            }
        });
        return output;
        """

        self.fill_answers_js = """
        (function(answerInput, targetScore) {
            console.log("接收到的答案字符串：", answerInput);
            console.log("目标分数：", targetScore);

            const answers = {};
            answerInput.trim().split('\\n').forEach(line => {
                const [id, answerPart] = line.split(' - ');
                if (id && answerPart) {
                    const answerValues = answerPart.split(',').map(ans => ans.trim());
                    answers[id.trim()] = answerValues;
                }
            });

            console.log("解析后的答案对象：", answers);

            const questions = Array.from(document.querySelectorAll('.kques-item[data-parentid=""]'));
            const totalQuestions = questions.length;

            if (isNaN(targetScore) || targetScore < 0 || targetScore > 100) {
                alert("目标分数无效，必须是 0 到 100 之间的整数。");
                return;
            }

            const correctQuestionsNeeded = Math.ceil((targetScore / 100) * totalQuestions);
            const wrongQuestionsNeeded = totalQuestions - correctQuestionsNeeded;

            console.log(`总题数: ${totalQuestions}, 需要答对: ${correctQuestionsNeeded}, 需要答错: ${wrongQuestionsNeeded}`);

            const questionIds = questions.map(question => question.getAttribute('data-id'));
            const wrongQuestionIds = [];

            if (wrongQuestionsNeeded > 0) {
                while (wrongQuestionIds.length < wrongQuestionsNeeded) {
                    const randomId = questionIds[Math.floor(Math.random() * questionIds.length)];
                    if (!wrongQuestionIds.includes(randomId)) {
                        wrongQuestionIds.push(randomId);
                    }
                }
            }

            questions.forEach(question => {
                const dataId = question.getAttribute('data-id');
                const correctAnswer = answers[dataId];

                if (correctAnswer) {
                    const questionType = question.getAttribute('type');
                    const shouldAnswerWrong = wrongQuestionIds.includes(dataId);

                    if (questionType === '1') {
                        const inputs = Array.from(question.querySelectorAll('input[type="checkbox"], input[type="radio"]'));

                        if (shouldAnswerWrong) {
                            const wrongInputs = inputs.filter(input => {
                                const optionText = input.closest('label').textContent.trim().replace(/^[A-Z]\.\s*/, '');
                                return !correctAnswer.map(ans => ans.toLowerCase()).includes(optionText.toLowerCase());
                            });
                            if (wrongInputs.length > 0) {
                                const randomWrongInput = wrongInputs[Math.floor(Math.random() * wrongInputs.length)];
                                randomWrongInput.checked = true;
                            } else {
                                inputs.forEach(input => input.checked = false);
                            }
                        } else {
                            inputs.forEach(input => {
                                const optionText = input.closest('label').textContent.trim().replace(/^[A-Z]\.\s*/, '');
                                if (correctAnswer.map(ans => ans.toLowerCase()).includes(optionText.toLowerCase())) {
                                    input.checked = true;
                                }
                            });
                        }
                    } else if (questionType === '2') {
                        const inputs = question.querySelectorAll('.gap-input, input[type="text"], textarea');

                        if (shouldAnswerWrong) {
                            inputs.forEach(input => {
                                input.value = " ";
                            });
                        } else {
                            correctAnswer.forEach((ans, idx) => {
                                if (inputs[idx]) {
                                    inputs[idx].value = ans;
                                }
                            });
                        }
                    }
                }
            });

            console.log("所有答案已成功填充！");

        })(arguments[0], arguments[1]);
        """

    def setup_browser(self):
        try:
            options = Options()
            options.add_argument('--no-sandbox')
            options.add_argument('--headless')
            options.add_argument('--disable-dev-shm-usage')
            options.add_argument('--disable-gpu')
            options.add_argument('--remote-debugging-port=9222')
            options.add_argument('--window-size=1920x1080')

            # 使用 webdriver_manager 自動下載對應版本的 ChromeDriver
            service = Service(ChromeDriverManager().install())

            self.driver = webdriver.Chrome(
                service=service,
                options=options
            )

            self.driver.set_window_size(1920, 1080)
            self.wait = WebDriverWait(self.driver, 20)
            print("瀏覽器初始化成功！")

        except Exception as e:
            import traceback
            print(f"瀏覽器初始化失敗，錯誤信息: {str(e)}")
            print("詳細錯誤追蹤:")
            print(traceback.format_exc())
            raise

    def login(self):
        self.driver.get("https://flipclass.stust.edu.tw/")
        try:
            account_input = self.wait.until(EC.visibility_of_element_located((By.NAME, "account")))
            account_input.clear()
            account_input.send_keys(self.account)

            password_input = self.wait.until(EC.visibility_of_element_located((By.NAME, "password")))
            password_input.clear()
            password_input.send_keys(self.password)

            login_button = self.wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@type='submit' and @data-role='form-submit']")))
            login_button.click()

            self.handle_keep_login()

            self.wait.until(EC.visibility_of_element_located(
                (By.XPATH, "//div[@class='caption underline' and text()='最近事件']")
            ))
            print("登入成功！")
            return True
        except Exception as e:
            print("登入時發生錯誤:", e)
            return False

    def handle_keep_login(self):
        try:
            keep_login_button = self.wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "keepLoginBtn")))
            keep_login_button.click()
            print("選擇保持登入。")
        except:
            print("未檢測到保持登入提示。")

    def navigate_to_course(self):
        course_url = f"https://flipclass.stust.edu.tw/course/exam/{self.course_id}"
        self.driver.get(course_url)
        print(f"已進入課程頁面：{course_url}")

        keep_focus_script = "window.onblur = function() { window.focus(); };"
        self.driver.execute_script(keep_focus_script)

    def extract_exam_and_answer_urls(self):
        try:
            element = self.wait.until(EC.element_to_be_clickable(
                (By.XPATH, "//a[span[text()='開始測驗' or text()='繼續測驗']]")))
            href = element.get_attribute('href')
            self.exam_page_url = urljoin(self.driver.current_url, href)
            print("考試按鈕鏈接（原始）：", self.exam_page_url)

            self.answer_page_url = self.modify_url(self.exam_page_url)
            print("答案頁面鏈接：", self.answer_page_url)
            return True
        except Exception as e:
            print("提取考試和答案 URL 時發生錯誤:", e)
            return False

    def modify_url(self, url):
        url_parts = urlparse(url)
        query_params = parse_qs(url_parts.query)
        query_params = {key: query_params[key] for key in query_params if key in ['key', 'title']}
        query_params['mode'] = 'view'
        new_query = urlencode(query_params, doseq=True)
        return urlunparse((url_parts.scheme, url_parts.netloc, url_parts.path, url_parts.params, new_query, url_parts.fragment))

    def load_exam_page(self):
        if self.exam_page_url:
            self.driver.get(self.exam_page_url)
            print("已返回考試頁面：", self.exam_page_url)
        else:
            print("無法導航到考試頁面，exam_page_url 未定義。")

    def extract_answers(self):
        try:
            result = self.driver.execute_script(self.extract_answers_js)
            print("提取的答案數據:\n", result)
            return result
        except Exception as e:
            print("提取答案時發生錯誤:", e)
            return None

    def wait_before_submit(self):
        total_seconds = round(self.answer_time * 60)
        print(f"\n開始等待作答時間: {self.answer_time:.1f}分鐘")

        for remaining in range(total_seconds, 0, -5):
            minutes = remaining // 60
            seconds = remaining % 60

            if remaining < 60:
                time_str = f"\r剩餘時間: {seconds}秒"
            else:
                time_str = f"\r剩餘時間: {minutes:02d}分{seconds:02d}秒"

            if remaining <= 30:
                time_str += " (即將結束!)"

            print(time_str, end='', flush=True)
            time.sleep(min(5, remaining))

        print("\n作答時間結束，準備提交...")

    def fill_answers_and_submit(self, answers):
        try:
            self.driver.execute_script(self.fill_answers_js, answers, self.target_score)
            print(f"所有答案已成功填充！目標分數：{self.target_score}")

            self.wait_before_submit()

            submit_button = self.wait.until(EC.element_to_be_clickable(
                (By.XPATH, "//div[@class='tool-submit']//button[@role='submit-exam']")))
            submit_button.click()
            print("已點擊交卷按鈕。")

            confirm_button = self.wait.until(EC.element_to_be_clickable(
                (By.XPATH, "//button[@data-bb-handler='confirm' and text()='交卷']")))
            confirm_button.click()
            print("已確認交卷。")

        except Exception as e:
            print("填充答案或交卷時發生錯誤:", e)

    def get_score(self):
        try:
            self.wait.until(EC.visibility_of_element_located(
                (By.XPATH, "//div[@class='infobar']//div[@class='score']//span[contains(@class, 'recordScore')]")
            ))
            score_element = self.driver.find_element(
                By.XPATH, "//div[@class='infobar']//div[@class='score']//span[contains(@class, 'recordScore')]"
            )
            fullmark_element = self.driver.find_element(
                By.XPATH, "//div[@class='infobar']//div[@class='score']//span[@class='fullmark']"
            )

            score = score_element.text.strip()
            fullmark = fullmark_element.text.strip()
            print(f"您的得分是: {score} {fullmark}")
        except Exception as e:
            print("提取分數時發生錯誤:", e)
            import traceback
            traceback.print_exc()

    def run(self):
        self.setup_browser()
        if not self.login():
            print("登入失敗，停止執行。")
            self.driver.quit()
            return

        self.navigate_to_course()
        if not self.extract_exam_and_answer_urls():
            print("無法提取測驗或答案頁面 URL。")
            self.driver.quit()
            return

        self.driver.get(self.answer_page_url)
        print("已進入答案頁面：", self.answer_page_url)

        try:
            self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "kques-item")))
        except Exception as e:
            print("答案頁面加載超時:", e)
            self.driver.quit()
            return

        answers = self.extract_answers()

        self.load_exam_page()

        try:
            self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "kques-item")))
        except Exception as e:
            print("考試頁面加載超時:", e)
            self.driver.quit()
            return

        if answers:
            self.fill_answers_and_submit(answers)
        else:
            print("未能提取答案，請檢查錯誤。")
            self.driver.quit()
            return

        try:
            self.wait.until(EC.visibility_of_element_located(
                (By.XPATH, "//div[@class='infobar']")
            ))
            self.get_score()
        except Exception as e:
            print("分數頁面加載超時或提取分數時發生錯誤:", e)
            import traceback
            traceback.print_exc()

        self.driver.quit()

def validate_account(account):
    return len(account) == 8

if __name__ == "__main__":
    while True:
        account = input("請輸入您的帳號 (8個字元): ").strip()
        if not validate_account(account):
            print("錯誤：帳號必須是8個字元！")
            continue

        password = input("請輸入您的密碼: ").strip()
        if not password:
            print("錯誤：密碼不能為空！")
            continue

        course_id = input("請輸入考試課程ID: ").strip()
        if not course_id:
            print("錯誤：課程ID不能為空！")
            continue

        while True:
            try:
                target_score = input("請輸入期望分數 (0-100): ").strip()
                target_score = float(target_score)
                if target_score < 0 or target_score > 100:
                    print("錯誤：分數必須在0到100之間！")
                    continue
                target_score = round(target_score)
                break
            except ValueError:
                print("錯誤：請輸入有效的數字！")
                continue

        while True:
            try:
                answer_time = input("請輸入作答時間(分鐘，支援小數點，例如0.5): ").strip()
                answer_time = float(answer_time)
                if answer_time <= 0:
                    print("錯誤：作答時間必須大於0分鐘！")
                    continue
                break
            except ValueError:
                print("錯誤：請輸入有效的數字！")
                continue

        print("\n=== 請確認您的輸入資訊 ===")
        print(f"帳號: {account}")
        print(f"密碼: {'*' * len(password)}")
        print(f"課程ID: {course_id}")
        print(f"期望分數: {target_score}分")
        print(f"作答時間: {answer_time:.1f}分鐘")

        confirm = input("\n確認是否正確？(y/n): ").strip().lower()
        if confirm == 'y':
            break
        print("\n請重新輸入資訊。\n")

    try:
        print("\n正在啟動自動化程序...")
        automation = FlipclassExamBot(account, password, course_id, answer_time, target_score)
        automation.run()
    except Exception as e:
        print(f"程序執行時發生錯誤: {e}")
        input("按Enter鍵結束程序...")